<a href="https://colab.research.google.com/github/RyEggGit/FCC-ML/blob/main/Book_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-03-28 01:54:00--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   104MB/s    in 0.2s    

2022-03-28 01:54:01 (104 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})


In [4]:
#Copy df_ratings
df_ratings_train = df_ratings

# Remove users that have less than 200 hundred Ratings 
- this saves RAM and lowers the likelyhood of users that give randoms ratings

In [5]:
ratings = df_ratings['user'].value_counts()
print(df_ratings['user'].isin(ratings[ratings <= 200].index).sum())
print(df_ratings['user'].isin(ratings[ratings >= 200].index).sum())

623424
527556


In [6]:
count_all = df_ratings['user'].value_counts()
vals_to_remove = count_all[count_all <= 200].index.values
print(len(vals_to_remove))
df_ratings_train['user'].loc[df_ratings_train['user'].isin(vals_to_remove)] = None
print(df_ratings_train['user'].isnull().sum())
df_ratings_train = df_ratings_train.dropna()

104384
623424


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


# Remove books that have less than 100 ratings
- this saves RAM and limits books that don't have many ratings

In [7]:
count_all = df_ratings['isbn'].value_counts()
vals_to_remove = count_all[count_all <= 100].index.values
print(len(vals_to_remove))
df_ratings_train['isbn'].loc[df_ratings_train['isbn'].isin(vals_to_remove)] = None
print(df_ratings_train['isbn'].isnull().sum())
df_ratings_train = df_ratings_train.dropna()

339835
477102


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


# Create Model

In [11]:
# Create 2D array that matches user to book and fill empty values with 0 (for dot product)
df_book_features = df_ratings_train.pivot(
    index='isbn',
    columns='user',
    values='rating'
).fillna(0)

In [12]:
df_book_features_matrix = csr_matrix(df_book_features.values)
model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model.fit(df_book_features_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

# Make predictions with model

In [13]:
#function to turn isbn into book name

def isbnToName(isbn):
  index = df_books['isbn'][df_books['isbn'] == isbn].index[0]
  return df_books[index: index+1]['title'].values[0]

In [14]:
# Function to turn name to isbn
def nameToIsbn(name):
  index = df_books['title'][df_books['title'] == name].index[0]
  return df_books[index: index+1]['isbn'].values[0]

In [15]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  book_isbn = nameToIsbn(book)
  index = np.where(df_book_features.index.values == book_isbn)[0][0]
  distances, indices = model.kneighbors(df_book_features.iloc[index,:].values.reshape(1, -1), n_neighbors = 6)

  recommended_books = pd.DataFrame({
      'title'   : df_book_features.index[indices.flatten()],
      'distance': distances[0],
    }).sort_values(by='distance', ascending=False).head(5)
  recommended_books['title'] = recommended_books['title'].apply(lambda x:isbnToName(x))
  return [book, recommended_books.values.tolist()]
get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [["I'll Be Seeing You", 0.8016210794448853],
  ['The Weight of Water', 0.7708583474159241],
  ['The Surgeon', 0.7699410915374756],
  ['I Know This Much Is True', 0.7677075266838074],
  ['The Lovely Bones: A Novel', 0.7230184078216553]]]

# Test

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [16]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7230184078216553]]]
You passed the challenge! 🎉🎉🎉🎉🎉
